# Laplacian Associative Memory (LAM) - DEV 26

[Multiscale representations of community structures in attractor neural networks](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8412329/pdf/pcbi.1009296.pdf) (2021)

- Simulation Render

In [1]:
from nn import LAM
import libtools

In [2]:
import PIL.Image, imageio
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

---

#### LOAD DATA

In [3]:
src = np.load('../data/mat/SOM_orient_v1.npy')

print('Map:', src.shape)
print('min:', f"{src.min():.2f}", 'max:', f"{src.max():.2f}")

Map: (201, 201)
min: 0.00 max: 6.28


In [4]:
img = np.array(PIL.Image.open("../data/images/coco/mesh.png").convert('L'), dtype=float)
img /= np.max(img)

print('Image:', img.shape)
print('min:', f"{img.min():.2f}", 'max:', f"{img.max():.2f}")

Image: (201, 201)
min: 0.05 max: 1.00


In [5]:
features = libtools.gabor_conv(img, src)

print('Features:', features.shape)
print('min:', f"{features.min():.2f}", 'max:', f"{features.max():.2f}")

Features: (201, 201)
min: 0.00 max: 1.84


---

### HYPER-PARAMETERS

In [6]:
factor = 4
src = libtools.downsample(src, factor)
features = libtools.downsample(features, factor)

In [8]:
N = 128**2                          # Neurons | Default: 30000
prob = 0.1                          # Sparsity
eta = 0.01                          # Epsilon / Learning rate
simlen = 3000                       # Steps / Epochs
gamma = 0.6                         # Inhibition ratio
norm_mode = 'asym'                  # Asymmetric Normalisation
alpha = -0.9                        # Ratio between local and global inhibition
P = src.shape[0] * src.shape[1]     # N Nodes

sigmaX = 0.0001                     # Spatial Gaussian Scale
sigmaA = 0.1                        # Angular Gaussian Scale

# Init condition (Center cell/pixel)
start_node = int(int(src.shape[0]/2) * src.shape[1] + int(src.shape[0]/2))

print('Neurons:', N)
print("Nodes:", P)
print('Start node:', start_node)
print(f"{P * 0.138:.2f} point attactors")

Neurons: 16384
Nodes: 2601
Start node: 1300
358.94 point attactors


---

### COMPUTE

In [ ]:
W = libtools.construct_SLAM(src, sigmaX, sigmaA) # Adjacency Matrix

In [ ]:
graph = LAM(N, P, prob, W, gamma, "asym") # Weights

In [ ]:
init_state = graph._set_state(features, 0.0415) # Initial Condition

[a, b] = libtools.unit_count(graph.xi[:, start_node], init_state)
print('A:', a)
print('B:', b)
print('Diff:', np.diff([a,b])[0])

In [ ]:
m_log, _ = graph.simulate_single(alpha, eta, simlen, init_state.copy(), cond=True) # Simulate

---

### RENDER

In [ ]:
seq = []
for i in range(simlen):
    if i%10==0:
        m = m_log[i,:]
        m = m + abs(m.min())
        m /= m.max()
        # m[start_node] = m.min()
        frame = m.reshape(src.shape)
        seq.append(plt.colormaps['magma'](frame) * 255.0)

frames = np.uint8(seq)
imageio.mimsave('../data/render/patternSim_spatial_initCond.gif', frames)